In [48]:
from pathlib import Path

root_dir = Path("data", "adaptation")
static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")
tif_files = list(hazard_path.glob("*.tif"))

In [49]:
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,HazardSection,)
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkSection)
from ra2ce.analysis.damages.damages import AnalysisSectionDamages
from ra2ce.analysis.analysis_config_data.analysis_config_data import (AnalysisConfigData,ProjectSection,AnalysisSectionBase,AnalysisSectionLosses)

In [50]:

_network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       
    primary_file = [network_path.joinpath("network.shp")], 
    file_id = "ID",
    link_type_column='highway',
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[Path(file) for file in hazard_path.glob("*.tif")],
    hazard_field_name= ['waterdepth'],
    aggregate_wl = AggregateWlEnum.MEAN,
    hazard_crs = 'EPSG:4326' 
)


_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=_network_section,
    hazard=_hazard
    )

In [51]:

handler = Ra2ceHandler.from_config(_network_config_data, None)

handler.configure()
handler.run_analysis()

100%|██████████| 460/460 [00:00<00:00, 456010.36it/s]
2024-12-03 11:56:20 AM - [avg_speed_calculator.py:176] - root - WARNING - No valid file found with average speeds data\adaptation\static\output_graph\avg_speed.csv, calculating and saving them instead.
2024-12-03 11:56:20 AM - [avg_speed_calculator.py:176] - root - WARNING - No valid file found with average speeds data\adaptation\static\output_graph\avg_speed.csv, calculating and saving them instead.
2024-12-03 11:56:20 AM - [avg_speed_calculator.py:176] - root - WARNING - No valid file found with average speeds data\adaptation\static\output_graph\avg_speed.csv, calculating and saving them instead.
2024-12-03 11:56:20 AM - [avg_speed_calculator.py:176] - root - WARNING - No valid file found with average speeds data\adaptation\static\output_graph\avg_speed.csv, calculating and saving them instead.
2024-12-03 11:56:20 AM - [avg_speed_calculator.py:176] - root - WARNING - No valid file found with average speeds data\adaptation\static\o

In [ ]:
from ra2ce.analysis.analysis_config_data.analysis_config_data import (AnalysisSectionAdaptation,AnalysisSectionAdaptationOption)
from ra2ce.analysis.analysis_config_data.enums.analysis_enum import AnalysisEnum
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import (
    AnalysisDamagesEnum
)
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum

damages_section = AnalysisSectionDamages(
    analysis=AnalysisDamagesEnum.DAMAGES,
    damage_curve=  DamageCurveEnum.HZ,
    #input_path = root_dir.joinpath("input_data")
)

losses_section = AnalysisSectionLosses(
    analysis=AnalysisLossesEnum.SINGLE_LINK_LOSSES,
    resilience_curves_file= root_dir.joinpath(
            "loss_functions", "resilience_curve.csv"),
    traffic_intensities_file= root_dir.joinpath(
            "loss_functions", "traffic_intensities.csv"),
    values_of_time_file= root_dir.joinpath(
            "loss_functions", "values_of_time.csv"
        ), 
)

adaptation_op0 = AnalysisSectionAdaptationOption(
    id="AO0",
    name="No adaptation",
)

adaptation_op1 = AnalysisSectionAdaptationOption(
    id="AO1",
    name="Cheap construction, expensive maintenance",
    construction_cost=1000,
    construction_interval=10,
    maintenance_interval=3,
    maintenance_cost=200,
)

adaptation_op2 = AnalysisSectionAdaptationOption(
    id="AO2",
    name="Expensive construction, cheap maintenance",
    construction_cost=5000,
    construction_interval=100,
    maintenance_interval=3,
    maintenance_cost=50,
)

adaptation_section = AnalysisSectionAdaptation(
    analysis=AnalysisEnum.ADAPTATION,
    losses_analysis=AnalysisLossesEnum.SINGLE_LINK_LOSSES,
    discount_rate=0.025,
    time_horizon=20,
    climate_factor=0.0,
    initial_frequency=0.0,
    #no_adaptation_option=AnalysisSectionAdaptationOption(adaptation_op0),
    adaptation_options=[adaptation_op0,adaptation_op1, adaptation_op2]
)

_analysis_config_data = AnalysisConfigData(
    analyses= [losses_section,damages_section,adaptation_section],
    root_path= root_dir,
    output_path= output_path,
    static_path= static_path,
    input_path= root_dir.joinpath("input"),
    )

In [ ]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)

#handler.configure()
handler.run_analysis()